# Tutorial: Extracting Information from Text Using OpenAI's API

This tutorial demonstrates how to use OpenAI's API to extract information from a text input. We will go through the steps of setting up the OpenAI client, making requests to the API, and processing the responses. The example involves summarizing a Danish news article and extracting structured information from it.

## Installation and Imports
First, we need to install the required library and import necessary modules.


In [ ]:
!pip install openai -q

In [2]:
# Import required libraries
from openai import OpenAI
from google.colab import userdata
import json
from pydantic import BaseModel, Field

Setting Up the OpenAI Client

We will set up the OpenAI client using a custom API key and base URL.

In [3]:
# Setup OpenAI client with custom API key and base URL
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

### Summarizing Text

We will call the language model to summarize a given Danish text into a single sentence.

In [4]:
# Create client
client = OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY
)

In [5]:
text = """
Spektakulær sølvskat fra vikingetiden fundet nord for Aarhus
En 22-årig arkæologistuderende står bag fundet.

Da den 22-årige arkæologistuderende Gustav Bruunsgaard i foråret var ude at gå med sin metaldetektor ved Elsted nord for Aarhus, gjorde han sig noget af et fund:

Først fandt han én armring i sølv.

Få dage efter vendte han tilbage til stedet og fandt yderligere seks armringe.

Både danske og internationale eksperter konkluderer, at de syv armringe stammer fra vikingetiden.

Det skriver Moesgaard Museum i en pressemeddelelse, hvor de kalder skatten spektakulær.

Elstedgårdskatten er et fantastisk interessant fund fra vikingetiden, siger Kasper H. Andersen, der er ph.d. og historiker på Moesgaard Museum, i pressemeddelelsen.

Trækker tråde til øst og vest

Fundet understreger ifølge Kasper H. Andersen, at Aarhus var et centralt knudepunkt i vikingetiden.

Ringene trækker nemlig tråde til både Rusland, Ukraine og De Britiske Øer, fremgår det af pressemeddelelse.

Da Gustav Bruunsgaards metaldetektor bippede, gravede han den første armring op med sin medbragte spade. På marken, hvor der var gevinst, er der før blevet fundet spor efter bebyggelse fra vikingetiden.

En af ringene - den oprullede - er en type, der oprindeligt kommer fra Rusland eller Ukraine. Den er siden blevet efterlignet her i Norden.

Tre af ringene, som er båndformede og stemplede, er en skandinavisk type, der siden gav inspiration til armringe i Irland.

De tre glatte armringe er sjældne, men er kendt fra Skandinavien og England, skriver Moesgaard Museum i pressemeddelelsen.

På den måde understreger fundet, hvordan Aarhus var et centralt knudepunkt i vikingernes verden, som gik helt fra Nordatlanten til Asien, siger Kasper H. Andersen.

Fungerede som betalingsmiddel

Eksperter har dateret armringene tilbage til 800-årene, som er den tidlige vikingetid. Det er også kort tid efter, at vikingetidens Aarhus blev grundlagt - det, der dengang blev kaldt Aros.

Dengang målte man værdi i sølv. Armringene var tilpasset et vægtsystem, så man kunne se de enkelte ringes værdi, og de blev brugt som betalingsmiddel. De demonstrerede også ejerens økonomiske formåen, skriver museet.

Hvis du vil se de syv armringe, kan de opleves nu på Moesgaard Museum. Herefter bliver de overdraget til Nationalmuseet.
"""

In [6]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    #model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",

    messages=[
        {
            "role": "system",
            "content": "Du er en hjælpsom assistent, der udtrækker information fra dansk tekst. Svar på dansk",
        },
        {
            "role": "user",
            "content": "Opsumer følgende til 1 kort sætning: " + text ,
        },
    ],
)

output = chat_completion.choices[0].message.content

In [ ]:
print(output)

## Creating a User Object and extracting structured info
We will define a schema for a user and call the API to create a user object based on this schema.

In [8]:
# Define the schema for the output.
class User(BaseModel):
    name: str = Field(description="user name")
    address: str = Field(description="address")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": User.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Create a user named Alice, who lives in 42, Wonderland Avenue.",
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

## Extracting Case Details
We will define a schema for case details and extract relevant information from the given text.

In [10]:
class NewsEventDetails(BaseModel):
    event_date: str = Field(description="Date when the event occurred or was reported")
    event_location: str = Field(description="Location where the event took place")
    reporter_name: str = Field(description="Name of the person reporting or involved in the event")
    reporter_age: int = Field(description="Age of the person reporting or involved in the event")
    event_description: str = Field(description="Description of the event")
    items_involved: list[str] = Field(description="Items or entities involved in the event")
    historical_context: str = Field(description="Historical or contextual background of the event")
    origins: list[str] = Field(description="Possible origins or sources related to the event")
    significance: str = Field(description="Importance or impact of the event")
    current_location: str = Field(description="Current location of items or entities involved")
    future_location: str = Field(description="Future location or outcome for items or entities involved")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": NewsEventDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Extract case informatin form following.: " + text,
        },
    ],
)



In [ ]:
extracted_output = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(extracted_output, ensure_ascii=False, indent=2))

In [14]:
json_schema = """{'properties': {'event_date': {'description': 'Date when the event occurred or was reported',
   'title': 'Event Date',
   'type': 'string'},
  'event_location': {'description': 'Location where the event took place',
   'title': 'Event Location',
   'type': 'string'},
  'reporter_name': {'description': 'Name of the person reporting or involved in the event',
   'title': 'Reporter Name',
   'type': 'string'},
  'reporter_age': {'description': 'Age of the person reporting or involved in the event',
   'title': 'Reporter Age',
   'type': 'integer'},
  'event_description': {'description': 'Description of the event',
   'title': 'Event Description',
   'type': 'string'},
  'items_involved': {'description': 'Items or entities involved in the event',
   'items': {'type': 'string'},
   'title': 'Items Involved',
   'type': 'array'},
  'historical_context': {'description': 'Historical or contextual background of the event',
   'title': 'Historical Context',
   'type': 'string'},
  'origins': {'description': 'Possible origins or sources related to the event',
   'items': {'type': 'string'},
   'title': 'Origins',
   'type': 'array'},
  'significance': {'description': 'Importance or impact of the event',
   'title': 'Significance',
   'type': 'string'},
  'current_location': {'description': 'Current location of items or entities involved',
   'title': 'Current Location',
   'type': 'string'},
  'future_location': {'description': 'Future location or outcome for items or entities involved',
   'title': 'Future Location',
   'type': 'string'}},
 'required': ['event_date',
  'event_location',
  'reporter_name',
  'reporter_age',
  'event_description',
  'items_involved',
  'historical_context',
  'origins',
  'significance',
  'current_location',
  'future_location'],
 'title': 'NewsEventDetails',
 'type': 'object'}"""

In [15]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "Du er en hjælpsom assistent, der udtrækker information fra dansk tekst som JSON. Kun JSON. Ingen forklaringer. Ingen ekstra info!!!",
        },
        {
            "role": "user",
            "content": "udtræk fra følgende tekst.: " + text + "Brug følgende JSON schema: " + json_schema,
        },
    ],
)



In [ ]:
extracted_output = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(extracted_output, ensure_ascii=False, indent=2))


In [18]:
text_2 = """

Massive demonstration i København: Tusindvis af mennesker samlet for klimaretfærdighed

Titusindvis af borgere gik i dag på gaden i København for at demonstrere for klimaretfærdighed. Demonstrationen, som blev arrangeret af en koalition af miljøorganisationer, fagforeninger og ungdomsbevægelser, er en af de største klimaprotester i Danmarks historie.

Demonstrationen startede på Rådhuspladsen, hvor taler fra forskellige aktivister, forskere og politikere satte fokus på behovet for øjeblikkelige handlinger mod klimaforandringer. Mange demonstranter bar skilte med budskaber som "Der er ingen planet B" og "Handle nu for fremtiden".

En af talerne var 17-årige Mette Nielsen, der har været aktiv i Fridays for Future-bevægelsen siden 2019. Hun understregede vigtigheden af, at de unge bliver hørt i klimadebatten.

Politiet anslår, at omkring 30.000 mennesker deltog i protesten. Demonstrationen forløb fredeligt, men med en stærk og højlydt stemning.

Demonstranterne afsluttede dagen med en march gennem byen, hvor de sluttede på Christiansborg Slotsplads, hvor flere politikere blev mødt med krav om mere ambitiøse klimaplaner.

Demonstrationen kommer i kølvandet på en uge med intense klimadebatter i Folketinget, hvor der er blevet diskuteret nye mål for CO2-reduktioner og bæredygtig energiomstilling.

Demonstranterne kræver, at regeringen og virksomhederne tager deres ansvar alvorligt og arbejder mod en grønnere og mere retfærdig fremtid.
"""

In [19]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "Du er en hjælpsom assistent, der udtrækker information fra dansk tekst som JSON. Kun JSON. Ingen forklaringer. Ingen ekstra info!!!",
        },
        {
            "role": "user",
            "content": "udtræk fra følgende tekst.: " + text + "Brug følgende JSON schema: " + json_schema,
        },
                {
            "role": "assistant",
            "content": """{
  "event_date": "-",
  "event_location": "Elsted, nord for Aarhus",
  "reporter_name": "Gustav Bruunsgaard",
  "reporter_age": 22,
  "event_description": "En 22-årig arkæologistuderende fandt en sølvskat med syv armringe på Elsted nord for Aarhus.",
  "items_involved": [
    "armringe",
    "metaldetektor",
    "spade"
  ],
  "historical_context": "vikingetiden",
  "origins": [
    "Rusland",
    "Ukraine",
    "De Britiske Øer",
    "Skandinavien",
    "Irland"
  ],
  "significance": "Fundet understreger Aarhus var et centralt knudepunkt i vikingetiden.",
  "current_location": "Moesgaard Museum",
  "future_location": "Nationalmuseet"
}""",
        },
                {
            "role": "user",
            "content": "udtræk fra følgende tekst.: " + text_2 + "Brug følgende JSON schema: " + json_schema,
        },
    ],
)



In [ ]:
chat_completion.choices[0].message.content

In [ ]:
extracted_output = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(extracted_output, ensure_ascii=False, indent=2))
